# LLM

Retrieval-Augmented Generation (RAG) with Large Language Models (LLMs) for document classification. The workflow leverages ChromaDB for vector storage and retrieval, Sentence Transformers for embedding, and Ollama for LLM inference.

## 1. Embedding and Storing Document Types
- Functionality: The code reads a JSON file (classification.json) containing document types and their descriptions. Each description is embedded using a Sentence Transformer model (all-MiniLM-L6-v2 by default).
- Storage: The embeddings, along with metadata, are stored in a ChromaDB collection called document_types. This enables efficient similarity search later.

## 2.  Retrieval-Augmented Classification
- Embedding Input: When a new document needs to be classified, its text is embedded using the same Sentence Transformer model.
- Retrieval: The embedding is used to query ChromaDB for the top-k most similar document type descriptions.
- Prompt Construction: The retrieved descriptions are formatted into a context string, which is then included in a prompt for the LLM.
- LLM Inference: The prompt is sent to an Ollama model (e.g., phi4-mini, llama3, gemma3, mistral:7b). The LLM is asked to classify the document based on the retrieved context and return only the document type.

## Sample data :  
- Classifcaiton 
```json
{
  "Invoice": "Contains billing details, amounts, due dates, sender/receiver info, itemized lists.",
  "Contract": "Includes legal terms, parties involved, signatures, obligations, clauses.",
  "Resume": "Lists work experience, education, skills, personal contact information.",
  "Email": "Contains sender, recipient, subject, body, and often a conversational tone.",
  "Report": "Summarizes findings, includes data analysis, conclusions, and recommendations.",
  "Letter": "Formal or informal communication, often includes a greeting, body, and closing.",
  "Presentation": "Visual slides with text, images, and charts to convey information or ideas.",
  "Proposal": "Outlines a plan or suggestion, often includes objectives, methods, and costs.",
  "Job Post": "Describes a job opening, including role, responsibilities, qualifications, location, and company details."
}
```

### inputfiles 
- file1.txt -   Resume 
- file2.txt -  Job Posting 
- file3.txt - Letter of Appointment 

In [1]:
import json
import chromadb
from sentence_transformers import SentenceTransformer

def embed_text(text, model="all-MiniLM-L6-v2"):
    """
    Embeds text using a sentence transformer model.
    """
    embedder = SentenceTransformer(model)
    return embedder.encode(text, convert_to_numpy=True).tolist()

def store_embeddings_in_chromadb(classification_json_path='data/classification.json', 
                                embedder_model="all-MiniLM-L6-v2", 
                                collection_name="document_types"):
    """
    Embeds document types from classification.json and stores them in ChromaDB.
    """
    # Initialize ChromaDB client
    client = chromadb.PersistentClient(path="./data/chromadb_data")
    
    # Create or get collection
    try:
        collection = client.get_or_create_collection(
            name=collection_name,
            metadata={"hnsw:space": "cosine"}  # Use cosine similarity
        )
    except Exception as e:
        print(f"Error creating collection: {e}")
        return

    # Load classification data
    with open(classification_json_path, 'r') as f:
        classification_context = json.load(f)
    
    # Prepare data for ChromaDB
    documents = []
    embeddings = []
    metadatas = []
    ids = []
    
    for idx, (doc_type, desc) in enumerate(classification_context.items()):
        embedding = embed_text(desc, embedder_model)
        documents.append(desc)
        embeddings.append(embedding)
        metadatas.append({"doc_type": doc_type})
        ids.append(f"doc_{idx}")
    
    # Store in ChromaDB
    try:
        collection.add(
            documents=documents,
            embeddings=embeddings,
            metadatas=metadatas,
            ids=ids
        )
        print(f"Successfully stored {len(documents)} embeddings in ChromaDB collection '{collection_name}'")
    except Exception as e:
        print(f"Error storing embeddings: {e}")

if __name__ == "__main__":
    store_embeddings_in_chromadb()

c:\new_works\nlp-projects\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Successfully stored 9 embeddings in ChromaDB collection 'document_types'


In [2]:
import chromadb
client = chromadb.PersistentClient(path="./data/chromadb_data")
# get the collection print collection name
collections = client.list_collections() 
for collection in collections:
    print(f"Collection Name: {collection.name}, Metadata: {collection.metadata}")
 

Collection Name: document_types, Metadata: {'hnsw:space': 'cosine'}


In [3]:
import ollama
import chromadb
from sentence_transformers import SentenceTransformer

def embed_text(text, model="all-MiniLM-L6-v2"):
    """
    Embeds text using a sentence transformer model.
    """
    embedder = SentenceTransformer(model)
    return embedder.encode(text, convert_to_numpy=True).tolist()

def retrieve_relevant_context(document_text, collection_name="document_types", 
                            embedder_model="all-MiniLM-L6-v2", top_k=3):
    """
    Retrieves the top_k most relevant document type descriptions from ChromaDB.
    """
    # Initialize ChromaDB client
    client = chromadb.PersistentClient(path="./data/chromadb_data")
    
    # Get collection
    try:
        collection = client.get_collection(name=collection_name)
    except Exception as e:
        print(f"Error accessing collection: {e}")
        return []

    # Embed input document
    doc_embedding = embed_text(document_text, embedder_model)
    
    # Query ChromaDB
    try:
        results = collection.query(
            query_embeddings=[doc_embedding],
            n_results=top_k,
            include=["documents", "metadatas", "distances"]
        )
    except Exception as e:
        print(f"Error querying ChromaDB: {e}")
        return []

    # Format results
    relevant_context = []
    for doc, metadata, distance in zip(results["documents"][0], results["metadatas"][0], results["distances"][0]):
        similarity = 1 - distance  # Convert distance to similarity (for cosine)
        relevant_context.append((metadata["doc_type"], doc, similarity))
    
    return relevant_context

def getModelResponse(document_text, model="phi4-mini", collection_name="document_types", top_k=3):
    """
    Gets a classification response from the Ollama model using RAG with ChromaDB.
    Returns the predicted document type.
    """
    # Retrieve relevant context from ChromaDB
    relevant_context = retrieve_relevant_context(document_text, collection_name, top_k=top_k)
    
    # Build context string from retrieved documents
    context_str = "Relevant Document Types and Descriptions:\n"
    for doc_type, desc, score in relevant_context:
        context_str += f"- {doc_type}: {desc} (Similarity: {score:.2f})\n"
    
    # Construct prompt with retrieved context
    prompt = (
        f"{context_str}\n"
        "Based on the above relevant document types, classify the following document and return only the type:\n\n"
        f"{document_text}"
    )

    # Get response from Ollama model
    response = ollama.chat(
        model=model,
        messages=[
            {'role': 'user', 'content': prompt},
        ]
    )
    doc_type = response['message']['content'].strip()

    return doc_type

In [4]:
import time

files = ['file1.txt', 'file2.txt', 'file3.txt']
models = ['phi4-mini', 'llama3', 'gemma3','mistral:7b']
# models = ['llama3']


for file in files:
    file_path = f'data/{file}'
    with open(file_path, 'r') as file:
        content = file.read().strip()

        for model in models:
            start_time = time.time()
            doctype = getModelResponse( content, collection_name="document_types", top_k=3, model=model)
            end_time = time.time()
            print(f"{model} Document Type: {doctype}  Execution Duration: {end_time - start_time:.2f} seconds")

        print(f"\t\t****** end o file process {file_path} *******\n")



phi4-mini Document Type: Job Post. (Similarity: 0.15)  Execution Duration: 11.20 seconds
llama3 Document Type: Resume  Execution Duration: 30.42 seconds
gemma3 Document Type: Resume  Execution Duration: 12.51 seconds
mistral:7b Document Type: Resume  Execution Duration: 34.09 seconds
		****** end o file process data/file1.txt *******

phi4-mini Document Type: Job Post  Execution Duration: 13.51 seconds
llama3 Document Type: Job Post  Execution Duration: 26.58 seconds
gemma3 Document Type: Job Post  Execution Duration: 11.93 seconds
mistral:7b Document Type: Job Post  Execution Duration: 28.30 seconds
		****** end o file process data/file2.txt *******

phi4-mini Document Type: Job Post  Execution Duration: 12.57 seconds
llama3 Document Type: Job Post  Execution Duration: 23.83 seconds
gemma3 Document Type: Job Post  Execution Duration: 9.95 seconds
mistral:7b Document Type: Job Post  Execution Duration: 24.06 seconds
		****** end o file process data/file3.txt *******



In [ ]:
import time

files = ['file1.txt', 'file2.txt', 'file3.txt']
models = ['phi4-mini', 'llama3', 'gemma3','mistral:7b']
# models = ['llama3']


for file in files:
    file_path = f'data/{file}'
    with open(file_path, 'r') as file:
        content = file.read().strip()

        for model in models:
            start_time = time.time()
            doctype = getModelResponse( content, collection_name="document_types", top_k=10  , model=model)
            end_time = time.time()
            print(f"{model} Document Type: {doctype}  Execution Duration: {end_time - start_time:.2f} seconds")

        print(f"\t\t****** end o file process {file_path} *******\n")



phi4-mini Document Type: Resume  Execution Duration: 20.36 seconds
llama3 Document Type: Resume  Execution Duration: 41.14 seconds
gemma3 Document Type: Resume  Execution Duration: 17.11 seconds
mistral:7b Document Type: The document provided is a Resume.  Execution Duration: 44.91 seconds
		****** end o file process data/file1.txt *******

phi4-mini Document Type: Job Post  Execution Duration: 17.39 seconds
llama3 Document Type: Job Post  Execution Duration: 37.30 seconds
gemma3 Document Type: Job Post  Execution Duration: 15.57 seconds
mistral:7b Document Type: The document type is "Job Post" (Similarity: 0.31)  Execution Duration: 40.11 seconds
		****** end o file process data/file2.txt *******

phi4-mini Document Type: Letter  Execution Duration: 16.77 seconds
llama3 Document Type: Letter  Execution Duration: 33.17 seconds
gemma3 Document Type: Job Post  Execution Duration: 14.21 seconds
mistral:7b Document Type: Job Post  Execution Duration: 35.39 seconds
		****** end o file proce